# Amazon Bedrock RAG 평가 도구를 사용한 Q&A 애플리케이션 구축 및 평가

## 개요
이 노트북에서는 Amazon Bedrock Knowledge Bases의 Retrieve API를 사용하여 Q&A 애플리케이션을 구축하고, Amazon Bedrock에서 제공하는 RAG 평가 도구를 사용하여 응답을 평가하는 방법을 다룹니다. 여기서는 유사성 검색을 기반으로 원하는 수의 문서 청크를 검색하고, Anthropic Claude를 사용하여 쿼리를 프롬프트하며, Amazon Bedrock의 RAG 평가 도구를 사용하여 응답을 효과적으로 평가합니다.

## Amazon Bedrock Knowledge Bases 소개
Knowledge Bases를 통해 Amazon Bedrock의 기반 모델(FM)을 회사 데이터에 안전하게 연결하여 검색 증강 생성(RAG)을 수행할 수 있습니다. 추가 데이터에 대한 액세스는 FM을 지속적으로 재훈련하지 않고도 모델이 더 관련성 있고 맥락에 특화된 정확한 응답을 생성하는 데 도움이 됩니다.

## 패턴
검색 증강 생성(RAG) 패턴을 사용하여 솔루션을 구현할 수 있습니다. RAG는 언어 모델 외부에서 데이터를 검색하고 관련 검색 데이터를 컨텍스트에 추가하여 프롬프트를 증강합니다.

## 사전 요구사항
질문에 답하기 전에 문서를 처리하고 Knowledge Base에 저장해야 합니다:
1. 문서(데이터 소스)를 Amazon S3 버킷에 업로드
2. Amazon Bedrock Knowledge Bases 생성
3. Knowledge Base ID 기록

## 설정
이 노트북을 실행하려면 종속성을 설치해야 합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install boto3 botocore langchain langchain-aws pandas tqdm --upgrade --quiet

In [ ]:
# 커널 재시작
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## 라이브러리 및 클라이언트 초기화

In [ ]:
# Knowledge Base ID 설정 (실제 Knowledge Base ID로 교체하세요)
%store -r kb_id
#kb_id = "<<your_knowledge_base_id>>"  # 여기에 실제 Knowledge Base ID를 입력하세요

import boto3
from botocore.client import Config

sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})

bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config)

print(f"Region: {region}, Account ID: {account_id}")

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import RetrievalQA

# 텍스트 생성용 LLM
llm_for_text_generation = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0", 
    client=bedrock_client
)

# 평가용 LLM
llm_for_evaluation = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", 
    client=bedrock_client
)

# 임베딩 모델
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    client=bedrock_client
)

## Retrieve API 설정
Amazon Bedrock Knowledge Bases의 Retrieve API를 사용하여 검색기를 생성합니다.

In [ ]:
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=kb_id,
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
)

## Q&A 체인 생성 및 테스트

In [ ]:
# 테스트 쿼리
query = "국내 인구 증감 양상을 분석하여 앞으로 어떻게 인구가 변화할지 예상해줘."

qa_chain = RetrievalQA.from_chain_type(
    llm=llm_for_text_generation, 
    retriever=retriever, 
    return_source_documents=True
)

response = qa_chain.invoke(query)
print("응답:")
print(response["result"])

## 평가 데이터 준비
Amazon Bedrock RAG 평가 도구를 사용하기 위한 평가 데이터셋을 준비합니다.

In [ ]:
import pandas as pd

questions = [
        "2024년 전국의 총인구와 전년 대비 증감률을 제시하고, 내국인과 외국인의 구성을 설명하시오.",
        "2024년 전국의 연령대별 인구 구성과 노령화지수를 제시하시오.",
        "2024년 다문화가구의 규모와 유형별 구성비를 설명하시오.",
        "2024년 (반)지하 및 옥탑 주택의 지역별 분포 특성을 설명하시오.",
        "2024년 전국 주택의 노후기간별 현황을 제시하시오."
]
ground_truths = [
    "2024년 총인구는 51,806천 명이며 전년 대비 0.1%(31천 명) 증가했습니다. 내국인: 49,763천 명(96.1%), 전년 대비 77천 명(-0.2%) 감소. 외국인: 2,043천 명(3.9%), 전년 대비 108천 명(5.6%) 증가",
    "2024년 연령대별 인구 구성은 다음과 같습니다: 유소년인구(0-14세): 542만 명(10.5%), 생산연령인구(15-64세): 3,626만 명(70.0%), 고령인구(65세 이상): 1,012만 명(19.5%), 노령화지수: 186.7(전년 대비 15.7 증가)",
    "2024년 다문화가구는 439,304가구이며, 유형별 구성은 다음과 같습니다: 귀화자가구: 42.6%, 결혼이민자가구: 35.2%, 다문화자녀가구: 11.3%, 기타: 10.9% 전년 대비 5.7%(23,720가구) 증가했습니다.",
    "2024년 기준: (반)지하가 있는 주택: 261천 호(전체 주택의 1.3%), 옥탑이 있는 주택: 34천 호(전체 주택의 0.2%), 수도권 분포 비율: (반)지하: 97.3%(254천 호) 옥탑: 90.6%(31천 호)",
    "2024년 주택의 노후기간별 현황: 20년 이상 된 주택: 10,908천 호(전체 주택의 54.9%) 20~30년 미만: 5,334천 호(26.8%) 30년 이상: 5,574천 호(28.0%)"
]

# 각 질문에 대한 답변과 컨텍스트 생성
answers = []
contexts = []

for query in questions:
    result = qa_chain.invoke(query)
    answers.append(result["result"])
    contexts.append([docs.page_content for docs in retriever.invoke(query)])

# 데이터셋 DataFrame 생성
dataset_df = pd.DataFrame({
    'questions': questions,
    'ground_truths': ground_truths,
    'answers': answers,
    'contexts': contexts
})

print(f"평가 데이터셋 준비 완료: {len(questions)}개 질문")

## Amazon Bedrock RAG 평가 도구를 사용한 평가
Amazon Bedrock에서 제공하는 RAG 평가 도구를 사용하여 응답을 평가합니다.

In [ ]:
import json
import re
import time
from tqdm.notebook import tqdm

def evaluate_with_bedrock_rag_tool(question, answer, context, ground_truth, metric_type):
    evaluation_prompt = f"""
    다음 RAG 시스템의 응답을 {metric_type} 관점에서 0-1 스케일로 평가해주세요.
    
    질문: {question}
    검색된 컨텍스트: {' '.join(context)}
    생성된 답변: {answer}
    정답: {ground_truth}
    
    0.0-1.0 사이의 점수만 반환해주세요.
    """
    
    try:
        # 요청 사이에 지연 시간 추가
        time.sleep(2)  # 2초 대기
        
        response = llm_for_evaluation.invoke(evaluation_prompt)
        score_text = response.content.strip()
        score_match = re.search(r'\b0?\.[0-9]+\b|\b1\.0\b|\b[01]\b', score_text)
        if score_match:
            return float(score_match.group())
        else:
            return 0.5
    except Exception as e:
        print(f"평가 중 오류 발생: {e}")
        # 오류 발생 시 더 긴 대기 시간 추가
        time.sleep(5)  # 5초 대기
        return 0.5

# 배치 크기 설정하여 한 번에 처리하는 양 제한
batch_size = 2

# 평가 메트릭 정의
metrics = {
    'faithfulness': 'faithfulness',
    'relevance': 'relevance', 
    'context_precision': 'context_precision',
    'context_recall': 'context_recall',
    'answer_correctness': 'answer_correctness'
}

evaluation_results = []

print("Amazon Bedrock RAG 평가 도구를 사용하여 평가 중...")

# 배치 단위로 처리
with tqdm(total=len(questions), desc="전체 평가 진행률") as pbar:
    for batch_start in range(0, len(questions), batch_size):
        batch_end = min(batch_start + batch_size, len(questions))
        batch_items = list(zip(questions[batch_start:batch_end], 
                          ground_truths[batch_start:batch_end],
                          answers[batch_start:batch_end], 
                          contexts[batch_start:batch_end]))
        
        for idx, (question, ground_truth, answer, context) in enumerate(batch_items, start=batch_start+1):
            result = {'sample_id': idx}
            for metric_name, metric_type in metrics.items():
                score = evaluate_with_bedrock_rag_tool(
                    question, answer, context, ground_truth, metric_type
                )
                result[metric_name] = score
            evaluation_results.append(result)
            pbar.update(1)

# 결과를 DataFrame으로 변환
evaluation_df = pd.DataFrame(evaluation_results)
print("\n평가 완료!")

## 평가 결과 분석

In [ ]:
# 평가 결과와 원본 데이터 결합
final_results = pd.concat([dataset_df.reset_index(drop=True), evaluation_df], axis=1)

# 결과 표시
print("=== Amazon Bedrock RAG 평가 결과 ===")
print("\n평가 메트릭별 평균 점수:")
for metric in metrics.keys():
    avg_score = evaluation_df[metric].mean()
    print(f"{metric}: {avg_score:.3f}")

print("\n상세 결과:")
display_columns = ['sample_id'] + list(metrics.keys())
print(evaluation_df[display_columns].to_string(index=False))

# 전체 결과 저장
final_results.to_csv('bedrock_rag_evaluation_results.csv', index=False, encoding='utf-8-sig')
print("\n결과가 'bedrock_rag_evaluation_results.csv' 파일로 저장되었습니다.")

In [ ]:
print("평가 데이터셋 상세 분석\n")
print("-" * 80)

for idx, (question, ground_truth, answer) in enumerate(zip(questions, ground_truths, answers), 1):
    print(f"\n샘플 #{idx}")
    print("\n질문:")
    print(question)
    print("\n정답 (Ground Truth):")
    print(ground_truth)
    print("\n모델 생성 답변:")
    print(answer)
    print("\n각 메트릭 점수:")
    sample_scores = evaluation_df[evaluation_df['sample_id'] == idx].iloc[0]
    for metric in metrics.keys():
        print(f"{metric}: {sample_scores[metric]:.2f}")
    print("-" * 80)

# 전체 평균 점수 표시
print("\n전체 평균 점수:")
for metric in metrics.keys():
    mean_score = evaluation_df[metric].mean()
    print(f"{metric}: {mean_score:.2f}")

## 평가 메트릭 설명

Amazon Bedrock RAG 평가 도구에서 사용하는 주요 메트릭들:

1. **Faithfulness (충실성)**: 생성된 답변이 주어진 컨텍스트와 얼마나 일치하는지 측정합니다. 0-1 범위이며, 높을수록 좋습니다.

2. **Relevance (관련성)**: 생성된 답변이 주어진 질문과 얼마나 관련성이 있는지 평가합니다. 불완전하거나 중복된 정보가 있는 답변은 낮은 점수를 받습니다.

3. **Context Precision (컨텍스트 정밀도)**: 검색된 컨텍스트에서 관련성 있는 항목들이 상위 순위에 있는지 평가합니다. 0-1 범위이며, 높을수록 좋습니다.

4. **Context Recall (컨텍스트 재현율)**: 검색된 컨텍스트가 정답과 얼마나 일치하는지 측정합니다. 0-1 범위이며, 높을수록 좋습니다.

5. **Answer Correctness (답변 정확성)**: 생성된 답변이 정답과 비교했을 때의 정확성을 평가합니다. 의미적 유사성과 사실적 유사성을 모두 고려합니다.

## 성능 최적화 권장사항

점수를 기반으로 RAG 워크플로우의 다른 구성 요소를 검토하여 점수를 더욱 최적화할 수 있습니다:

- 청킹 전략 검토
- 프롬프트 지침 개선
- 추가 컨텍스트를 위한 numberOfResults 증가
- 임베딩 모델 변경 고려
- 검색 구성 매개변수 조정

**참고**: 위의 점수는 RAG 애플리케이션의 성능에 대한 상대적인 아이디어를 제공하며, 신중하게 사용해야 하고 독립적인 점수로 사용해서는 안 됩니다. 또한 평가를 위해 5개의 질문/답변 쌍만 사용했으므로, 모범 사례로는 모델 평가를 위해 문서의 다양한 측면을 다루는 충분한 데이터를 사용해야 합니다.

## 정리

비용 발생을 방지하기 위해 Knowledge Base, OpenSearch 인덱스 및 관련 IAM 역할과 정책을 삭제하는 것을 잊지 마세요.